In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('payroll.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        designation TEXT,
        salary REAL
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS payroll_records (
        id INTEGER PRIMARY KEY,
        employee_id INTEGER,
        month TEXT,
        year INTEGER,
        amount REAL,
        FOREIGN KEY (employee_id) REFERENCES employees (id)
    )
''')

# Add a new employee
def add_employee():
    name = input("Enter employee name: ")
    designation = input("Enter employee designation: ")
    salary = float(input("Enter employee salary: "))

    cursor.execute('''
        INSERT INTO employees (name, designation, salary)
        VALUES (?, ?, ?)
    ''', (name, designation, salary))
    conn.commit()
    print('Employee added successfully.')

    # Get the generated employee ID
    cursor.execute('SELECT last_insert_rowid()')
    employee_id = cursor.fetchone()[0]
    print("Employee ID:", employee_id)

# View employee details
def view_employee_details():
    choice = input("View employee details by:\n1. ID\n2. Name\n")
    if choice == '1':
        employee_id = int(input("Enter employee ID: "))

        cursor.execute('SELECT * FROM employees WHERE id = ?', (employee_id,))
        employee = cursor.fetchone()

        if not employee:
            print("Employee not found.")
            return

        print("Employee ID:", employee[0])
        print("Name:", employee[1])
        print("Designation:", employee[2])
        print("Salary:", employee[3])

    elif choice == '2':
        name = input("Enter employee name: ")

        cursor.execute('SELECT * FROM employees WHERE name = ?', (name,))
        employee = cursor.fetchone()

        if not employee:
            print("Employee not found.")
            return

        print("Employee ID:", employee[0])
        print("Name:", employee[1])
        print("Designation:", employee[2])
        print("Salary:", employee[3])

    else:
        print("Invalid choice. Please try again.")

# Delete an employee
def delete_employee():
    employee_id = int(input("Enter employee ID to delete: "))

    cursor.execute('SELECT * FROM employees WHERE id = ?', (employee_id,))
    employee = cursor.fetchone()

    if not employee:
        print("Employee not found.")
        return

    cursor.execute('DELETE FROM employees WHERE id = ?', (employee_id,))
    conn.commit()
    print("Employee deleted successfully.")

# Edit employee details
def edit_employee():
    employee_id = int(input("Enter employee ID to edit: "))

    cursor.execute('SELECT * FROM employees WHERE id = ?', (employee_id,))
    employee = cursor.fetchone()

    if not employee:
        print("Employee not found.")
        return

    name = input("Enter employee name: ")
    designation = input("Enter employee designation: ")
    salary = float(input("Enter employee salary: "))

    cursor.execute('''
        UPDATE employees SET name = ?, designation = ?, salary = ?
        WHERE id = ?
    ''', (name, designation, salary, employee_id))
    conn.commit()
    print("Employee details updated successfully.")

# Add payroll record for an employee
def add_payroll_record():
    employee_id = int(input("Enter employee ID: "))

    cursor.execute('SELECT * FROM employees WHERE id = ?', (employee_id,))
    employee = cursor.fetchone()

    if not employee:
        print("Employee not found.")
        return

    month = input("Enter month: ")
    year = int(input("Enter year: "))
    amount = float(input("Enter amount: "))

    cursor.execute('''
        INSERT INTO payroll_records (employee_id, month, year, amount)
        VALUES (?, ?, ?, ?)
    ''', (employee_id, month, year, amount))
    conn.commit()
    print('Payroll record added successfully.')

# View salary details
# View salary details for an employee
def view_salary_details():
    employee_id = int(input("Enter employee ID: "))

    cursor.execute('SELECT * FROM employees WHERE id = ?', (employee_id,))
    employee = cursor.fetchone()

    if not employee:
        print("Employee not found.")
        return

    month_from = input("Enter start month (e.g., May): ")
    month_to = input("Enter end month (e.g., September): ")

    cursor.execute('''
        SELECT month, year, amount FROM payroll_records
        WHERE employee_id = ? AND month BETWEEN ? AND ?
    ''', (employee_id, month_from, month_to))
    records = cursor.fetchall()

    if not records:
        print("No salary records found for the given range of months.")
        return

    total_salary = 0
    print("Month\tYear\tAmount")
    for record in records:
        print(f"{record[0]}\t{record[1]}\t{record[2]}")
        total_salary += record[2]

    cursor.execute('''
        SELECT COUNT(*) FROM payroll_records
        WHERE employee_id = ? AND month BETWEEN ? AND ?
    ''', (employee_id, month_from, month_to))
    num_of_records = cursor.fetchone()[0]

    total_salary += num_of_records * employee[3]

    print("Total Salary:", total_salary)

# View all payroll records
def view_all_payroll_records():
    cursor.execute('SELECT * FROM payroll_records')
    records = cursor.fetchall()

    if not records:
        print("No payroll records found.")
        return

    print("Employee ID\tMonth\tYear\tAmount")
    for record in records:
        print(f"{record[1]}\t{record[2]}\t{record[3]}\t{record[4]}")

# Menu
while True:
    print("\n--- Payroll Management System ---")
    print("1. Add Employee")
    print("2. View Employee Details")
    print("3. Delete Employee")
    print("4. Edit Employee Details")
    print("5. Add Payroll Record")
    print("6. View Salary Details")
    print("7. View All Payroll Records")
    print("0. Exit")

    choice = input("Enter your choice: ")

    if choice == '1':
        add_employee()
    elif choice == '2':
        view_employee_details()
    elif choice == '3':
        delete_employee()
    elif choice == '4':
        edit_employee()
    elif choice == '5':
        add_payroll_record()
    elif choice == '6':
        view_salary_details()
    elif choice == '7':
        view_all_payroll_records()
    elif choice == '0':
        break
    else:
        print("Invalid choice. Please try again.")

# Close the database connection
conn.close()


--- Payroll Management System ---
1. Add Employee
2. View Employee Details
3. Delete Employee
4. Edit Employee Details
5. Add Payroll Record
6. View Salary Details
7. View All Payroll Records
0. Exit
Enter your choice: 2
View employee details by:
1. ID
2. Name
2
Enter employee name: john
Employee ID: 1
Name: john
Designation: ss
Salary: 1000.0

--- Payroll Management System ---
1. Add Employee
2. View Employee Details
3. Delete Employee
4. Edit Employee Details
5. Add Payroll Record
6. View Salary Details
7. View All Payroll Records
0. Exit
Enter your choice: 0
